In [3]:
task_num="4"
var_num=1
env_step_limit=2
num_episodes=1
gpt_model="gpt_4_0613"
summarize_end_of_episode=1
device="cpu"
temperature=0.0
use_gold_memory_in_ep0=0
gold_traces=""
use_last_k_memories=3
quadrant=1
simplifications_preset="easy"
output_path_prefix = 'logs/testrun/'

In [ ]:
jar_path = None
task_num = "4"
var_num = 1
env_step_limit = 100
num_episodes = 2
gpt_model = "gpt-4-0613"
summarize_end_of_episode = 1
device = None
temperature = 0.0
use_gold_memory_in_ep0 = 0
gold_traces = ""
use_last_k_memories = 3
quadrant = None
output_path_prefix = "save-histories"
simplifications_preset = None


In [4]:
task_num = int(task_num)
var_num = int(var_num)
print(f"Running CLIN agent for taskIdx:{task_num}, varIdx:{var_num}")

Running CLIN agent for taskIdx:4, varIdx:1


In [6]:
exitCommands = ["quit", "exit"]
from sentence_transformers import SentenceTransformer
sent_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens', device=device)


/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/agential/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# no gold path -_-

In [9]:
summaryfname = "logs/" + "summary.txt"
summaryFile = open(summaryfname, "w")

In [10]:
task = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring' 
print("Task Names: " + str(task))

Task Names: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring


In [11]:
"""CRITIC structured output module."""

from typing import Any, Dict, List

from pydantic import BaseModel, Field

from agential.agents.base.output import BaseAgentOutput
from agential.core.llm import Response


class ClinStepOutput(BaseModel):
    """Critic step Pydantic output class.

    Attributes:
        answer (str): The answer generated by the agent.
        critique (str): The critique of the answer generated by the agent.
        external_tool_info (Dict[str, Any]): The query requested by the agent.
        answer_response (List[Response]): The answer responses generated by the agent.
        critique_response (List[Response]): The critique responses generated by the agent.
    """

    query: str = Field(..., description="The query requested by the agent.")
    answer: str = Field(..., description="The answer generated by the agent.")
    observation: str = Field(..., description="The answer's observation.")
    external_tool_info: Dict[str, Any] = Field(
        ..., description="The external tool outputs."
    )
    answer_response: List[Response] = Field(
        ..., description="The answer responses generated by the agent."
    )
    observation_response: List[Response] = Field(
        ..., description="The observation responses generated by the agent."
    )


class ClinOutput(BaseAgentOutput):
    """Critic Pydantic output class.

    Attributes:
        additional_info (List[CriticStepOutput]): The additional info.
    """

    additional_info: List[ClinStepOutput] = Field(
        ..., description="The additional info."
    )


In [ ]:
class ReActQAStrategy(ReActGeneralStrategy):
    """A strategy class for QA benchmarks using the ReAct agent.

    Attributes:
        llm (BaseLLM): The language model used for generating answers and critiques.
        max_steps (int): The maximum number of steps the agent can take.
        max_tokens (int): The maximum number of tokens allowed for a response.
        enc (Encoding): The encoding used for the language model.
        docstore (DocstoreExplorer): The document store used for searching and looking up information.
        testing (bool): Whether the strategy is in testing mode. Defaults to False.
    """

    def __init__(
        self,
        llm: BaseLLM,
        max_steps: int = 6,
        max_tokens: int = 5000,
        enc: Encoding = tiktoken.encoding_for_model("gpt-3.5-turbo"),
        docstore: DocstoreExplorer = DocstoreExplorer(Wikipedia()),
        testing: bool = False,
    ) -> None:
        """Initialization."""
        super().__init__(
            llm=llm,
            max_steps=max_steps,
            max_tokens=max_tokens,
            enc=enc,
            testing=testing,
        )
        self.docstore = docstore

In [ ]:
from agential.agents.base.agent import BaseAgent
from agential.core.llm import BaseLLM

class Clin(BaseAgent):
    """Clin Agent.

    Attributes:
        llm (BaseLLM): An instance of a language model used for generating initial answers
            and critiques.
        benchmark (str): The benchmark.
        testing (bool): Whether to run in testing mode. Defaults to False.
        **strategy_kwargs (Any): Additional strategy-specific arguments.
    """

    def __init__(
        self,
        llm: BaseLLM,
        benchmark: str,
        testing: bool = False,
        **strategy_kwargs: Any,
    ) -> None:
        """Initialization."""
        super().__init__(llm=llm, benchmark=benchmark, testing=testing)

        self.strategy = Clin.get_strategy(
            benchmark=self.benchmark, llm=self.llm, testing=testing, **strategy_kwargs
        )


In [ ]:
num_test_variations_to_run = 1
for 